In [ ]:
!pip install -U tensorflow==2.15.1 #13 14

In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import os
import pathlib
import tensorflow as tf
import keras

from keras.utils import  Sequence
from keras.models import Sequential
from tensorflow.keras.utils import img_to_array, load_img, to_categorical
#models
#from efficientnet_lite import EfficientNetLiteB0, EfficientNetLiteB1, EfficientNetLiteB3
from tensorflow.keras.applications import EfficientNetB1, EfficientNetB0
#preproces
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from PIL import Image
import cv2 as cv
import matplotlib.pyplot as plt

from collections import Counter
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, accuracy_score

import subprocess
from IPython.display import FileLink, display

from sklearn.utils.class_weight import compute_class_weight

from keras import backend as K


from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = False

from sklearn.utils.class_weight import compute_class_weight
import shutil
import tqdm
tf.__version__, #keras.__version__

In [ ]:
data_directory = '/kaggle/input/mushrooms-classification-common-genuss-images/Mushrooms'
working_directory = '/kaggle/working/Mushrooms'

In [ ]:
if os.path.exists(working_directory):
    shutil.rmtree(working_directory)
os.makedirs(working_directory, exist_ok=True)

for class_name in tqdm.tqdm(os.listdir(data_directory),total=len(os.listdir(data_directory))):
    class_directory = os.path.join(data_directory, class_name)
    writable_class_directory = os.path.join(working_directory, class_name)
    os.makedirs(writable_class_directory, exist_ok=True)

    for image in os.listdir(class_directory):
        src_path = os.path.join(class_directory, image)
        dest_path = os.path.join(writable_class_directory, image)
        shutil.copy(src_path, dest_path)

In [ ]:
imgs = glob(r'/kaggle/working/Mushrooms/*/*')

SIZE = 245 #345 #414

for img_path in tqdm.tqdm(imgs, total=len(imgs)):
    try:
        Image.open(img_path).load()
    except Exception  as e:
        print(f'image file - {img_path} is truncated')
        imgs.remove(img_path)

labels = [el.split('/')[4] for el in imgs ]

len(imgs), len(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.3, random_state=91, stratify=labels)

train = pd.DataFrame(list(zip(X_train,y_train)), columns =['X','y'])
test = pd.DataFrame(list(zip(X_test,y_test)), columns =['X','y'])

datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=45,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True, 
        #brightness_range=(1.05, 0.9),
)
valid_gen = ImageDataGenerator(rescale=1./255)

training_generator = datagen.flow_from_dataframe(train, x_col='X', y_col="y", target_size=(SIZE, SIZE), shuffle=False,batch_size=16) #shuffle=False batch_size=16
validation_generator = valid_gen.flow_from_dataframe(test, x_col='X', y_col="y", target_size=(SIZE, SIZE), shuffle=False, batch_size=16) #interpolation='lanczos'

In [ ]:
training_generator.class_indices,


In [ ]:
validation_generator.class_indices

In [ ]:
validation_generator.batch_size

In [ ]:
test['y'].unique()

In [ ]:
test['y'].value_counts()

In [ ]:
train['y'].unique()

In [ ]:
train['y'].value_counts()

In [ ]:
def build_model(num_classes=9):
    base_model = EfficientNetB0( #B3
        input_shape=(SIZE,SIZE, 3),
        include_top=False,
        pooling="avg",
        weights="imagenet"
    )

    base_model.trainable=True

    return tf.keras.Sequential([
        base_model,
        
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.6), #0.6 0.7
        tf.keras.layers.Dense(210, activation="silu", kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l1(0.033)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax", kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l1(0.033))
    ])

model = build_model()


cls_weight = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)

loss = tf.keras.losses.CategoricalFocalCrossentropy(
    alpha=cls_weight,
    gamma=2.0,
    #from_logits=True,
    label_smoothing=0.0,
    
)

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model.compile(loss=loss, metrics=['accuracy'], optimizer=optimizer, )
early_stop = EarlyStopping(monitor='val_loss', verbose=1, patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=0, min_lr=1.0000e-08) #0



In [ ]:

#cls_weight = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
def build_model(num_classes=9, shape=(SIZE,SIZE,3), alpha=0.5):
    inputs = tf.keras.layers.Input(shape=shape)
    model = EfficientNetB1(include_top=False,
        weights='imagenet',
        input_tensor=inputs, # !
        input_shape=None,
        pooling=None,
        #classes=num_classes,
        classifier_activation='softmax')
    
    model.trainable = True

    # Rebuild top
    x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Dropout(0.5, name="top_dropout")(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNetB1")
    #loss = tf.keras.losses.CategoricalFocalCrossentropy(
    #    alpha=alpha,
     #   gamma=2,
     #   from_logits=False)
    loss = tf.keras.losses.CategoricalCrossentropy()

    
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

    
    model.compile(loss=loss, metrics=['accuracy'], optimizer=optimizer)
    
    
    return model


model = build_model()

early_stop = EarlyStopping(monitor='val_loss', verbose=1, patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=0, min_lr=1.0000e-08) 

In [ ]:


epoch = 5 #
history = model.fit(training_generator, validation_data=validation_generator, epochs=epoch, batch_size=16, callbacks=[early_stop, reduce_lr]  ) #callbacks=[early_stop, reduce_lr] 

In [ ]:
def data_prepare(image_path, labels, class_dict, n_clases=9, target_size=(345, 345)):
    #X = np.array([])
    X = np.empty([len(image_path), target_size[0],target_size[1], 3])
    for i, img in enumerate(image_path): #tqdm(total=len(image_path))
        img = load_img(img, target_size=target_size)
        img_array = img_to_array(img)
        
        X[i,] = img

  
    y = [label.replace(key, str(value)) 
                for label in  labels
                for key, value in class_dict.items() 
                if key in label]
        
    y =  keras.utils.to_categorical(y, num_classes=n_clases)
    return X, y



In [ ]:
def show_history(history):
    plt.figure(figsize=(18,8))

    plt.suptitle('Loss and Accuracy Plots', fontsize=18)

    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.xlabel('Number of epochs', fontsize=15)
    plt.ylabel('Loss', fontsize=15)

    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.xlabel('Number of epochs', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.show()

show_history(history)

In [ ]:
X_test_2, y_test_2 = data_prepare(X_test, y_test, training_generator.class_indices, target_size=(245, 245),)
predictions_2 = model.predict(X_test_2, )

In [ ]:
training_generator = datagen.flow_from_dataframe(train, x_col='X', y_col="y", target_size=(SIZE, SIZE), interpolation='lanczos', shuffle=False) #shuffle=False
validation_generator = valid_gen.flow_from_dataframe(test, x_col='X', y_col="y", target_size=(SIZE, SIZE),interpolation='lanczos', shuffle=False) #interpolation='lanczos'

In [ ]:
y_train = train['y']
with pd.option_context("future.no_silent_downcasting", True):
    y_train = y_train.replace(training_generator.class_indices).infer_objects(copy=False)


#y_train = y_train.replace(training_generator.class_indices)
y_train = keras.utils.to_categorical(y_train, num_classes=9)

In [ ]:




y_test = test['y']
with pd.option_context("future.no_silent_downcasting", True):
   y_test = y_test.replace(training_generator.class_indices).infer_objects(copy=False)
#y_test = y_test.replace(validation_generator.class_indices)
y_test = keras.utils.to_categorical(y_test, num_classes=9)


In [ ]:
predictions = model.predict(validation_generator, )

In [ ]:
model.evaluate(validation_generator)

In [ ]:

predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

print(classification_report(true_classes, predicted_classes))

cm = confusion_matrix(true_classes, predicted_classes)


plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()



In [ ]:

predictions3 = model.predict(X, )


In [ ]:
predictions2 = model.predict(training_generator, )

In [ ]:

predicted_classes = np.argmax(predictions2, axis=1)
true_classes = np.argmax(y_train, axis=1)

print(classification_report(true_classes, predicted_classes))

cm = confusion_matrix(true_classes, predicted_classes)


plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
predicted_classes = np.argmax(predictions_2, axis=1)
true_classes = np.argmax(y_test, axis=1)

print(classification_report(true_classes, predicted_classes))

cm = confusion_matrix(true_classes, predicted_classes)


plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
